In [1]:
import pandas as pd
from pyxlsb import open_workbook, convert_date
import re 
from rapidfuzz import process
from rapidfuzz import fuzz


file_path = '../../data/ecommerce_sales.xlsb'
sheet_name = '20210614 Ecommerce sales'

In [2]:
def read_xlsb(file_path, sheet_name):
    data = []
    with open_workbook(file_path) as wb:
        with wb.get_sheet(sheet_name) as sheet:
            for idx,row in enumerate(sheet.rows()):
                if idx == 0:
                    data.append([item.v for item in row])
                else:
                    data.append([item.v if item.c != 5 else convert_date(item.v) for item in row ])
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

In [3]:
df = read_xlsb(file_path, sheet_name)

df.head()

,Cod_cmd,Libellé produit,Vendeur,Univers,Nature,Date de commande,Montant cmd,Quantité,Prix transport,Délai transport annoncé
0,182210782.0,Table basse carrée detroit design industriel,Autre vendeur,Canapé Salon Séjour,Table basse,2021-01-20,244.0,4.0,6.67,10.0
1,182082437.0,Ours en peluche géant 150 cm brun,Autre vendeur,Enfant Bébé,Peluche,2021-01-17,28.0,1.0,9.92,10.0
2,182095765.0,Ours en peluche géant 100 cm blanc,Autre vendeur,Enfant Bébé,Peluche,2021-01-18,15.0,1.0,9.92,10.0
3,182615392.0,Lot de 4 chaises mia noires pour salle à manger,Autre vendeur,Canapé Salon Séjour,Chaise,2021-01-23,385.0,2.0,20.75,10.0
4,184222081.0,Meuble tv falko bois blanc et gris,Autre vendeur,Canapé Salon Séjour,Meuble tv,2021-02-11,61.0,1.0,19.08,10.0


In [6]:
# Predefined colors in French
FRENCH_COLORS = set([
    "rouge", "bleu", "vert", "jaune", "noir", "blanc", "orange", "rose", "violet", "marron", "gris", 
    "beige", "turquoise", "or", "argent", "ivoire", "bleu marine", "sarcelle", "bordeaux", "lavande", 
    "pêche", "citron vert", "corail", "cyan", "magenta", "olive", "saumon", "moutarde", "chocolat", 
    "charbon", "indigo", "rubis", "émeraude", "saphir", "doré", "argenté", "brun"
])

class ProductInfoExtractor:
    def __init__(self):
        self.dimension_pattern = re.compile(r'(\d+(?:[.,]\d+)?)\s*(?:x|×)\s*(\d+(?:[.,]\d+)?)\s*(?:cm|m)?') # 120 cm x 120 cm - > 120, 120
        self.single_dim_pattern = re.compile(r'(\d+(?:[.,]\d+)?)\s*(?:cm|m)') # 120 cm -> 120
        self.word_split_pattern = re.compile(r'\b\w+\b')
        #self.color_patterns = {color: re.compile(rf'\b{color}(?:s|es|e)?\b') for color in FRENCH_COLORS} 

    def extract_info(self, description):
        if not description:
            return "N/F", "N/F"
        
        desc_lower = str(description).lower()
        dimensions = self.extract_dimensions(desc_lower)
        colors = self.extract_colors(desc_lower)
        return dimensions, colors

    def extract_dimensions(self, desc_lower):
        dimensions_match = self.dimension_pattern.findall(desc_lower)
        if dimensions_match:
            dim1, dim2 = dimensions_match[0]
            dim1 = float(dim1.replace(',', '.'))
            dim2 = float(dim2.replace(',', '.'))
            return f"{dim1:.0f}x{dim2:.0f}"
        else:
            single_dim_match = self.single_dim_pattern.findall(desc_lower)
            if single_dim_match:
                dim1 = float(single_dim_match[0].replace(',', '.'))
                return f"{dim1:.0f}"
            else:
                return "N/F"
    
    def find_matching_colors(self, desc_lower):
        words = desc_lower.split()
        matching_words = []
        for word in words:
            if any(color in word.lower() for color in FRENCH_COLORS):  
                matching_words.append(word)
        
        return matching_words
    
    def extract_colors(self, desc_lower):
        colors = []
        matching_words = self.find_matching_colors(desc_lower)
        for word in matching_words:
            sim = process.extractOne(word, FRENCH_COLORS, scorer=fuzz.ratio, score_cutoff=75) 
            if sim:
                colors.append(sim[0])
        return ", ".join(colors) if colors else "N/F"
            
# Usage with a DataFrame
extractor = ProductInfoExtractor()
df[['Dimensions', 'Color']] = df['Libellé produit'].apply(lambda x: pd.Series(extractor.extract_info(x)))


In [7]:
# Display the results
dim_col = df[['Cod_cmd','Libellé produit', 'Dimensions', 'Color']]
dim_col

,Cod_cmd,Libellé produit,Dimensions,Color
0,182210782.0,Table basse carrée detroit design industriel,N/F,N/F
1,182082437.0,Ours en peluche géant 150 cm brun,150,brun
2,182095765.0,Ours en peluche géant 100 cm blanc,100,blanc
3,182615392.0,Lot de 4 chaises mia noires pour salle à manger,N/F,noir
4,184222081.0,Meuble tv falko bois blanc et gris,N/F,"blanc, gris"
...,...,...,...,...
525029,186997203.0,Lit coffre madrid 140x200 + 1 sommier ch ne bl...,140x200,blanc
525030,187045930.0,Nappe antitache ronde infroissable et 100 poly...,160,bleu
525031,187019687.0,Good morning housse de couette pour enfants da...,140x200,N/F
525032,186990574.0,Ampoule pour four e14 25w 230v ac,N/F,N/F


In [8]:
col_nf = dim_col[dim_col['Color'] == "N/F"]
col_nf

,Cod_cmd,Libellé produit,Dimensions,Color
0,182210782.0,Table basse carrée detroit design industriel,N/F,N/F
11,182074852.0,tagère à double position tania bois imitation...,161,N/F
12,182056932.0,tagère à double position tania bois imitation...,161,N/F
15,181586231.0,Console vintage leoni motifs graphiques,N/F,N/F
27,181916701.0,Bibliothèque vintage noemi bois pied épingle,N/F,N/F
...,...,...,...,...
525021,211132468.0,Piscine tubulaire 2 13 x 1 52 x 0 61m summer w...,13x1,N/F
525027,211135009.0,Réfrigérateur 2 portes 207l froid statique oce...,55,N/F
525028,187030040.0,Plaque induction oceanic 5500w 60cm oceati3z7,60,N/F
525031,187019687.0,Good morning housse de couette pour enfants da...,140x200,N/F


In [11]:
col_f = dim_col[dim_col['Color'] != "N/F"]
col_f

,Cod_cmd,Libellé produit,Dimensions,Color
1,182082437.0,Ours en peluche géant 150 cm brun,150,brun
2,182095765.0,Ours en peluche géant 100 cm blanc,100,blanc
3,182615392.0,Lot de 4 chaises mia noires pour salle à manger,N/F,noir
4,184222081.0,Meuble tv falko bois blanc et gris,N/F,"blanc, gris"
5,182603100.0,Meuble tv falko bois blanc et gris,N/F,"blanc, gris"
...,...,...,...,...
525025,211099572.0,Tristar mx 4804 robot patissier blanc,N/F,blanc
525026,211098105.0,Table basse wonderland table basse extensible ...,60,noir
525029,186997203.0,Lit coffre madrid 140x200 + 1 sommier ch ne bl...,140x200,blanc
525030,187045930.0,Nappe antitache ronde infroissable et 100 poly...,160,bleu


In [9]:
dimension_found = (df['Dimensions'] != "N/F").sum()
color_found = (df['Color'] != "N/F").sum()
total_products = len(df)

print(f"\nDimensions found in {dimension_found} out of {total_products} products ({dimension_found/total_products:.2%})")
print(f"Colors found in {color_found} out of {total_products} products ({color_found/total_products:.2%})")


Dimensions found in 227840 out of 525034 products (43.40%)
Colors found in 111265 out of 525034 products (21.19%)


In [10]:
both_found = df[(df['Dimensions'] != "N/F") & (df['Color'] != "N/F")]
print(f"\nProducts with both dimension and color found: {len(both_found)}")
print(both_found[['Libellé produit', 'Dimensions', 'Color']])


Products with both dimension and color found: 49499
                                          Libellé produit Dimensions  \
1                       Ours en peluche géant 150 cm brun        150   
2                      Ours en peluche géant 100 cm blanc        100   
13      Table à manger georgia 6 personnes blanche et ...        140   
14      Table à manger georgia 6 personnes blanche et ...        140   
16      Table à manger georgia 6 personnes blanche et ...        140   
...                                                   ...        ...   
525014  Gregory solide et confortable sommier tapissie...    140x200   
525019                        Canisse roseau naturel 1x5m        1x5   
525026  Table basse wonderland table basse extensible ...         60   
525029  Lit coffre madrid 140x200 + 1 sommier ch ne bl...    140x200   
525030  Nappe antitache ronde infroissable et 100 poly...        160   

              Color  
1              brun  
2             blanc  
13      blanc, n